In [ ]:
from pathlib import Path
from google.colab import drive

drive.mount('/content/gdrive')

root_dir = Path('/content/gdrive/MyDrive/Data')

Mounted at /content/gdrive


In [ ]:
!pip install mne
!pip install import_ipynb
!pip install modin

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 25.6 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.


In [ ]:
import os
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import import_ipynb

import torch
from torch import nn
import mne

In [ ]:
data_path = root_dir / Path("Sleep EDF Data/Telemetry")
files = list(data_path.iterdir())

data = mne.io.read_raw_edf(files[0])
res = np.array(data.get_data())

Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7151J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [ ]:
def rotate_and_reverse(arr):
  arr_rotated = np.rot90(arr)
  return arr_rotated[::-1]

res_roated_reversed = rotate_and_reverse(res)

In [ ]:
res.shape, res_roated_reversed.shape

((5, 3851000), (3851000, 5))

In [ ]:
data.ch_names

['EEG Fpz-Cz', 'EEG Pz-Oz', 'EOG horizontal', 'EMG submental', 'Marker']

In [ ]:
file_names = dict()

In [ ]:
def extract_key(file_name):
  key = file_name.split("/")
  return key[-1][:7], key[-1][9:12] == 'PSG'

for file in files:
  key, is_PSG = extract_key(str(file))
  if key not in file_names:
    file_names[key] = {"PSG": None, "Annotation": None}
  if is_PSG:
    file_names[key]["PSG"] = file
  else:
    file_names[key]["Annotation"] = file

In [ ]:
def load_edf_file(file_name, is_PSG=True):
  res = None
  if is_PSG:
    data = mne.io.read_raw_edf(file_name)
    res = data
    #res = np.array(data.get_data())
  else:
    data = mne.read_annotations(file_name)
    res = data
  return res

In [ ]:
# Removing bad and unaligned files
file_names.pop("ST7081J")
file_names.pop("ST7172J")
file_names.pop("ST7072J")
file_names.pop("ST7171J")
file_names.pop("ST7201J")
file_names.pop("ST7121J")
file_names.pop("ST7221J")

{'PSG': PosixPath('/content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7221J0-PSG.edf'),
 'Annotation': PosixPath('/content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7221JA-Hypnogram.edf')}

In [ ]:
def is_continuous(annotation):
  for i in range(1, len(annotation)):
    prev = annotation[i-1]
    onset = 0
    if prev['onset'] != None:
      onset = prev['onset']
    if prev['duration']+ onset != annotation[i]['onset']:
      return False
  return True

def check_dictionary_files(file_names):
  for file_name in file_names:
    annot = load_edf_file(file_names[file_name]["Annotation"], False)
    res = is_continuous(annot)
    print(f"{file_name}: {is_continuous(annot)}")

In [ ]:
check_dictionary_files(file_names)

ST7151J: True
ST7132J: True
ST7092J: True
ST7212J: True
ST7061J: True
ST7122J: True
ST7131J: True
ST7071J: True
ST7012J: True
ST7112J: True
ST7102J: True
ST7182J: True
ST7152J: True
ST7022J: True
ST7181J: True
ST7082J: True
ST7041J: True
ST7042J: True
ST7052J: True
ST7242J: True
ST7101J: True
ST7162J: True
ST7011J: True
ST7062J: True
ST7111J: True
ST7191J: True
ST7192J: True
ST7202J: True
ST7091J: True
ST7021J: True
ST7241J: True
ST7141J: True
ST7161J: True
ST7211J: True
ST7051J: True
ST7142J: True
ST7222J: True


In [ ]:
def convert_psg_to_dataframe(res_data):
  d = {'EEG Fpz-Cz': res_data[0], 'EEG Pz-Oz': res_data[1], 'EOG horizontal': res_data[2], 'EMG submental': res_data[3]}
  df = pd.DataFrame(data=d)
  return df

def annotation_info(annot):
  total_time = 0
  onset = 0
  if annot[0]['onset'] != None:
    onset = annot[0]['onset']
  for item in annot:
    total_time = total_time + item['duration']
  return onset, total_time

def convert_annotation_to_dataframe(annot, window=30):
  labels = []
  for item in annot:
    duration = item['duration']
    desc = item['description']
    label = [desc for i in range(int(duration/30))]
    labels.extend(label)
  d = {"labels":np.array(labels)}
  df = pd.DataFrame(data=d)
  return df

def process_file_names(file_names, number_files = 12):
  files = list(file_names.values())[:number_files]
  psgs = []
  annots = []
  for i in range(0, number_files, 5):
    psgs_temp = []
    annots_temp = []
    for j in range(min(number_files-i, 5)):
      file = files[i+j]
      psg = load_edf_file(file["PSG"]).get_data()
      psg_df = convert_psg_to_dataframe(psg)
      annot = load_edf_file(file["Annotation"], False)
      onset, total_time = annotation_info(annot)
      annot_df = convert_annotation_to_dataframe(annot)
      psg_df = psg_df.iloc[int(onset * 100): int((onset + total_time) * 100)]
      psgs_temp.append(psg_df)
      annots_temp.append(annot_df)
    psg_temp_df = pd.concat(psgs_temp, axis=0)
    psg_temp_df = psg_temp_df.reset_index(drop=True)
    annot_temp_df = pd.concat(annots_temp, axis=0)
    annot_temp_df = annot_temp_df.reset_index(drop=True)
    psgs.append(psg_temp_df)
    annots.append(annot_temp_df)
  psg_df = pd.concat(psgs, axis=0)
  psg_df = psg_df.reset_index(drop=True)
  annot_df = pd.concat(annots, axis=0)
  annot_df = annot_df.reset_index(drop=True)
  return psg_df, annot_df

In [ ]:
psg_df, annot_df = process_file_names(file_names, number_files = 20)

Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7151J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7132J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7092J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7212J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/gdrive/MyDrive/Data/Sleep EDF Data/Telemetry/ST7061J0-PSG.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from /content/gdrive/MyDrive/Data/

In [ ]:
save_psg = root_dir / Path("Sleep EDF Data/Processed/PSG.pkl")
save_annot = root_dir / Path("Sleep EDF Data/Processed/Annot.pkl")

psg_df.shape, annot_df.shape

((57930000, 4), (19310, 1))

In [ ]:
psg_df.to_pickle(save_psg)
annot_df.to_pickle(save_annot)